In [1]:
{-#LANGUAGE GADTs#-}
{-#LANGUAGE QuasiQuotes#-}

import Data.Tree
import Data.Foldable
import Text.Megaparsec
import Text.Megaparsec.Char
import qualified Text.Megaparsec.Char.Lexer as L
import Data.Void
import Data.String.QQ

In [2]:
type Parser = Parsec Void String

In [3]:
sc :: Parser ()
sc = L.space
  space1
  (L.skipLineComment "//")
  (empty)

lexeme = L.lexeme sc

symbol = L.symbol sc

braces = between (symbol "{") (symbol "}")

stringLiteral :: Parser String
stringLiteral = char '"' >> manyTill L.charLiteral (char '"')

nodeParser :: Parser (Forest String)
nodeParser = (\s -> [Node s []]) <$> lexeme stringLiteral

parseKey :: Parser (Forest String) -> Parser (Tree String)
parseKey rest = Node <$> lexeme stringLiteral <*> rest

treeParser :: Parser (Tree String)
treeParser = sc *> parseKey (nodeParser <|> braces (many treeParser))

In [4]:
parseTest stringLiteral "\"cheese\""

"cheese"

In [5]:
parseTest (parseKey nodeParser) "\"aa\"  \"bbb\""

Node {rootLabel = "aa", subForest = [Node {rootLabel = "bbb", subForest = []}]}

In [6]:
tree = parseMaybe treeParser "\"cheese\" \"pops\""

In [7]:
:t tree

tree :: Maybe (Tree String)

In [8]:
let Just t = tree

In [9]:
putStrLn $ drawTree t

cheese
|
`- pops

In [10]:
test = [s|
// HLTV overview description file for de_vertigo.bsp

"de_vertigo"
{
	"material"	"overviews/de_vertigo_radar"	// texture file
	"pos_x"		"-3168"	// upper left world coordinate
	"pos_y"		"1762"
	"scale"		"4.0" 

	"verticalsections"
	{
		"default" // use the primary radar image
		{
			"AltitudeMax" "20000"
			"AltitudeMin" "11700"
		}
		"lower" // i.e. de_nuke_lower_radar.dds
		{
			"AltitudeMax" "11700"
			"AltitudeMin" "-10000"
		}
	}

	// loading screen icons and positions
	"CTSpawn_x"	"0.54"
	"CTSpawn_y"	"0.25"
	"TSpawn_x"	"0.20"
	"TSpawn_y"	"0.75"

	"bombA_x"	"0.72"
	"bombA_y"	"0.60"
	"bombB_x"	"0.245"
	"bombB_y"	"0.27"
}

|] :: String

In [11]:
Just res = parseMaybe treeParser test

In [12]:
putStrLn $ drawTree res

de_vertigo
|
+- material
|  |
|  `- overviews/de_vertigo_radar
|
+- pos_x
|  |
|  `- -3168
|
+- pos_y
|  |
|  `- 1762
|
+- scale
|  |
|  `- 4.0
|
+- verticalsections
|  |
|  +- default
|  |  |
|  |  +- AltitudeMax
|  |  |  |
|  |  |  `- 20000
|  |  |
|  |  `- AltitudeMin
|  |     |
|  |     `- 11700
|  |
|  `- lower
|     |
|     +- AltitudeMax
|     |  |
|     |  `- 11700
|     |
|     `- AltitudeMin
|        |
|        `- -10000
|
+- CTSpawn_x
|  |
|  `- 0.54
|
+- CTSpawn_y
|  |
|  `- 0.25
|
+- TSpawn_x
|  |
|  `- 0.20
|
+- TSpawn_y
|  |
|  `- 0.75
|
+- bombA_x
|  |
|  `- 0.72
|
+- bombA_y
|  |
|  `- 0.60
|
+- bombB_x
|  |
|  `- 0.245
|
`- bombB_y
   |
   `- 0.27

In [13]:
altSubtree f t@(Node n l)
  | f n = pure t
  | otherwise = asum $ fmap (altSubtree f) l 

In [14]:
import Data.List
putStrLn $ drawForest $ altSubtree ("bomb" `isPrefixOf`) res

bombA_x
|
`- 0.72

bombA_y
|
`- 0.60

bombB_x
|
`- 0.245

bombB_y
|
`- 0.27

In [15]:
type Pos = (Double, Double)
type MinMax = (Int,Int)
type Scale = Double
data MapData = MapData Pos Scale deriving Show

import qualified Data.Map as M

data Data = Data {
    name :: String
  , material :: String
  , mapData :: MapData
  , verticalSections :: M.Map String MinMax
  , iconLocations :: M.Map String Pos
  } deriving Show

In [16]:
dropFromEnd s n = take (length s - n) s
dropFromEnd "cheese" 2

"chee"

In [17]:
import Text.Read
import Data.Maybe

treeLookup :: String -> Tree String -> Maybe String
treeLookup s t = do
    Node _ [Node res []] <- altSubtree (== s) t
    return res

getPositions :: Tree String -> M.Map String Pos
getPositions t = M.fromList $ catMaybes $ fmap findY xs
 where
     xs = altSubtree ("_x" `isSuffixOf`) t
     findY :: Tree String -> Maybe (String, Pos)
     findY (Node s [Node x []]) = do 
        let suffixless = dropFromEnd s 2
        y <- treeLookup (suffixless ++ "_y") t
        pos <- (,) <$> readMaybe x <*> readMaybe y
        return (suffixless , pos)
     findY _ = Nothing

In [18]:
getPositions res

fromList [("CTSpawn",(0.54,0.25)),("TSpawn",(0.2,0.75)),("bombA",(0.72,0.6)),("bombB",(0.245,0.27)),("pos",(-3168.0,1762.0))]

In [19]:
getVerticalSections :: Tree String -> M.Map String MinMax
getVerticalSections (Node _ sections) = M.fromList $ catMaybes secList
    where
      secList = map (\sub@(Node sec _) -> do
                        min <- treeLookup "AltitudeMin" sub >>= readMaybe
                        max <- treeLookup "AltitudeMax" sub >>= readMaybe
                        return (sec, (min,max))
                ) sections

In [20]:
convert :: Tree String -> Maybe Data
convert t = do
    let Node name _ = t
    material <- treeLookup "material" t
    let positions = getPositions t
    pos <- M.lookup "pos" positions
    scale <- treeLookup "scale" t >>= readMaybe
    verticalSections <- getVerticalSections <$> altSubtree (== "verticalsections") t
    let iconLocations = M.delete "pos" positions
    let mapData = MapData pos scale
    return $ Data name material mapData verticalSections iconLocations

In [21]:
convert res

Just (Data {name = "de_vertigo", material = "overviews/de_vertigo_radar", mapData = MapData (-3168.0,1762.0) 4.0, verticalSections = fromList [("default",(11700,20000)),("lower",(-10000,11700))], iconLocations = fromList [("CTSpawn",(0.54,0.25)),("TSpawn",(0.2,0.75)),("bombA",(0.72,0.6)),("bombB",(0.245,0.27))]})